In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Final_df_1.xls')

In [ ]:
df.shape

(211218, 20)

In [ ]:
df.head()

,User_Id,Coin,Execution_Price,Size_Tokens,Total_Amount,Buy/Sell,Timestamp,TimeofDay,Start_Position,Direction,PnL,Crossed,Fee,Date,Setinment_Index,Fear/Greed,Coin_Type,TradeResult,Hour,DayOfWeek
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,buy,2024-12-02 22:50:00,night,0.000000,open long,0.0,True,0.345404,2024-12-02,80,extreme greed,moderate,nill,22,Monday
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,buy,2024-12-02 22:50:00,night,986.524596,open long,0.0,True,0.005600,2024-12-02,80,extreme greed,moderate,nill,22,Monday
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,buy,2024-12-02 22:50:00,night,1002.518996,open long,0.0,True,0.050431,2024-12-02,80,extreme greed,moderate,nill,22,Monday
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,buy,2024-12-02 22:50:00,night,1146.558564,open long,0.0,True,0.050043,2024-12-02,80,extreme greed,moderate,nill,22,Monday
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,buy,2024-12-02 22:50:00,night,1289.488521,open long,0.0,True,0.003055,2024-12-02,80,extreme greed,moderate,nill,22,Monday


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104402 entries, 44 to 211217
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   User_Id          104402 non-null  int64  
 1   Coin             104402 non-null  object 
 2   Execution_Price  104402 non-null  float64
 3   Size_Tokens      104402 non-null  float64
 4   Total_Amount     104402 non-null  float64
 5   Buy/Sell         104402 non-null  object 
 6   Timestamp        104402 non-null  object 
 7   TimeofDay        104402 non-null  object 
 8   Start_Position   104402 non-null  float64
 9   Direction        104402 non-null  object 
 10  PnL              104402 non-null  float64
 11  Crossed          104402 non-null  bool   
 12  Fee              104402 non-null  float64
 13  Date             104402 non-null  object 
 14  Setinment_Index  104402 non-null  int64  
 15  Fear/Greed       104402 non-null  object 
 16  Coin_Type        104402 non-null  object 


In [ ]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
df['User_Id']= label_encoder.fit_transform(df['User_Id'])

df['User_Id'].unique()

array([25, 16, 13,  0,  9, 20, 12, 17,  8, 27,  1, 14, 29, 26,  5, 23, 21,
       19, 10,  7,  3,  6, 18, 24, 28,  4, 31, 11, 22, 30,  2, 15])

In [ ]:
df.columns

Index(['User_Id', 'Coin', 'Execution_Price', 'Size_Tokens', 'Total_Amount',
       'Buy/Sell', 'Timestamp', 'TimeofDay', 'Start_Position', 'Direction',
       'PnL', 'Crossed', 'Fee', 'Date', 'Setinment_Index', 'Fear/Greed',
       'Coin_Type', 'TradeResult', 'Hour', 'DayOfWeek'],
      dtype='object')

# ***Statistical analysis***

In [ ]:
from scipy.stats import f_oneway, shapiro, levene
# Ensure TimeofDay is categorical
df['TimeofDay'] = df['TimeofDay'].astype('category')

# Group data
groups = [grp['PnL'].values for _, grp in df.groupby('TimeofDay') if len(grp) >= 3]

# ✅ Assumption 1: Shapiro-Wilk Test for Normality
print("📌 Shapiro-Wilk Normality Test (PnL by TimeofDay):")
for name, grp in df.groupby('TimeofDay'):
    if len(grp) >= 3:
        stat, p = shapiro(grp['PnL'])
        print(f"  {name}: W = {stat:.3f}, p = {p:.3f} → {'✅ Normal' if p > 0.05 else '❌ Not Normal'}")

# ✅ Assumption 2: Levene’s Test for Homogeneity of Variance
stat_levene, p_levene = levene(*groups)
print(f"\n📌 Levene’s Test for Equal Variances: Stat = {stat_levene:.3f}, p = {p_levene:.3f} → {'✅ Equal variances' if p_levene > 0.05 else '❌ Unequal variances'}")

# ✅ One-Way ANOVA Test
f_stat, p_val = f_oneway(*groups)
print(f"\n📊 One-Way ANOVA: F = {f_stat:.3f}, p = {p_val:.4f}")
print("🔍 Result:", "✅ Significant difference in mean profit and loss across TimeofDay" if p_val < 0.05 else "❌ No significant difference in mean PnL across TimeofDay")


📌 Shapiro-Wilk Normality Test (PnL by TimeofDay):
  afternoon: W = 0.032, p = 0.000 → ❌ Not Normal
  evening: W = 0.060, p = 0.000 → ❌ Not Normal
  lateNight: W = 0.058, p = 0.000 → ❌ Not Normal
  morning: W = 0.062, p = 0.000 → ❌ Not Normal
  night: W = 0.050, p = 0.000 → ❌ Not Normal

📌 Levene’s Test for Equal Variances: Stat = 5.935, p = 0.000 → ❌ Unequal variances

📊 One-Way ANOVA: F = 7.630, p = 0.0000
🔍 Result: ✅ Significant difference in mean profit and loss across TimeofDay


/tmp/ipython-input-100-4116138344.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = [grp['PnL'].values for _, grp in df.groupby('TimeofDay') if len(grp) >= 3]
/tmp/ipython-input-100-4116138344.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for name, grp in df.groupby('TimeofDay'):
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 41556.
  res = hypotest_fun_out(*samples, **kwds)
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: s

In [ ]:
from scipy.stats import chi2_contingency

# Create a contingency table of 'Buy/Sell' and 'Fear/Greed'
contingency_table = pd.crosstab(df['Buy/Sell'], df['Fear/Greed'])

# Print the contingency table
print("Contingency Table:")
display(contingency_table)

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results of the chi-square test
print("\nChi-square test results:")
print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p:.4f}")
print(f"Degrees of freedom: {dof}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p < alpha:
    print(f"The p-value ({p:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between Buy/Sell decisions and Fear/Greed categories.")
else:
    print(f"The p-value ({p:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between Buy/Sell decisions and Fear/Greed categories.")

Contingency Table:


Fear/Greed,extreme fear,extreme greed,fear,greed,neutral
Buy/Sell,,,,,
buy,10935,17940,30270,24576,18969
sell,10465,22052,31567,25727,18717



Chi-square test results:
Chi-square statistic: 327.2355
P-value: 0.0000
Degrees of freedom: 4

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between Buy/Sell decisions and Fear/Greed categories.


In [ ]:
# Create a contingency table of 'Crossed' and 'Fear/Greed'
contingency_table_crossed_fear_greed = pd.crosstab(df['Crossed'], df['Fear/Greed'])

# Print the contingency table
print("Contingency Table (Crossed vs. Fear/Greed):")
display(contingency_table_crossed_fear_greed)

# Perform the chi-square test
chi2_crossed_fear_greed, p_crossed_fear_greed, dof_crossed_fear_greed, expected_crossed_fear_greed = chi2_contingency(contingency_table_crossed_fear_greed)

# Print the results of the chi-square test
print("\nChi-square test results (Crossed vs. Fear/Greed):")
print(f"Chi-square statistic: {chi2_crossed_fear_greed:.4f}")
print(f"P-value: {p_crossed_fear_greed:.4f}")
print(f"Degrees of freedom: {dof_crossed_fear_greed}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p_crossed_fear_greed < alpha:
    print(f"The p-value ({p_crossed_fear_greed:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between Crossed status and Fear/Greed categories.")
else:
    print(f"The p-value ({p_crossed_fear_greed:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between Crossed status and Fear/Greed categories.")

Contingency Table (Crossed vs. Fear/Greed):


Fear/Greed,extreme fear,extreme greed,fear,greed,neutral
Crossed,,,,,
False,9267,15108,24680,19393,14367
True,12133,24884,37157,30910,23319



Chi-square test results (Crossed vs. Fear/Greed):
Chi-square statistic: 225.4653
P-value: 0.0000
Degrees of freedom: 4

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between Crossed status and Fear/Greed categories.


In [ ]:
from scipy import stats

# Prepare data for ANOVA: create a list of Total_Amount for each Fear/Greed category
fear_greed_groups = df.groupby('Fear/Greed')['Total_Amount'].apply(list)

# Perform one-way ANOVA test
# Note: ANOVA requires that groups have roughly equal variances and the data within each group is approximately normally distributed.
# We proceed with the test, but these assumptions should ideally be checked.
f_statistic, p_value = stats.f_oneway(*fear_greed_groups)

# Print the results
print("ANOVA test results for Total_Amount across Fear/Greed categories:")
print(f"F-statistic: {f_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpret the results
alpha = 0.05  # Significance level

print("\nInterpretation:")
if p_value < alpha:
    print(f"The p-value ({p_value:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant difference in the mean Total_Amount across the Fear/Greed categories.")
else:
    print(f"The p-value ({p_value:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant difference in the mean Total_Amount across the Fear/Greed categories.")

ANOVA test results for Total_Amount across Fear/Greed categories:
F-statistic: 108.2960
P-value: 0.0000

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant difference in the mean Total_Amount across the Fear/Greed categories.


In [ ]:

# Create a contingency table of 'TimeofDay' and 'Fear/Greed'
contingency_table_timeofday_fear_greed = pd.crosstab(df['TimeofDay'], df['Fear/Greed'])

# Print the contingency table
print("Contingency Table (TimeofDay vs. Fear/Greed):")
display(contingency_table_timeofday_fear_greed)

# Perform the chi-square test
chi2_timeofday_fear_greed, p_timeofday_fear_greed, dof_timeofday_fear_greed, expected_timeofday_fear_greed = chi2_contingency(contingency_table_timeofday_fear_greed)

# Print the results of the chi-square test
print("\nChi-square test results (TimeofDay vs. Fear/Greed):")
print(f"Chi-square statistic: {chi2_timeofday_fear_greed:.4f}")
print(f"P-value: {p_timeofday_fear_greed:.4f}")
print(f"Degrees of freedom: {dof_timeofday_fear_greed}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p_timeofday_fear_greed < alpha:
    print(f"The p-value ({p_timeofday_fear_greed:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between TimeofDay and Fear/Greed categories.")
else:
    print(f"The p-value ({p_timeofday_fear_greed:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between TimeofDay and Fear/Greed categories.")

Contingency Table (TimeofDay vs. Fear/Greed):


Fear/Greed,extreme fear,extreme greed,fear,greed,neutral
TimeofDay,,,,,
afternoon,3669,9751,13094,8582,6460
evening,4000,7965,12192,11306,10559
lateNight,7186,11083,19364,18954,11059
morning,3606,7554,10713,6726,7277
night,2939,3639,6474,4735,2331



Chi-square test results (TimeofDay vs. Fear/Greed):
Chi-square statistic: 4197.7419
P-value: 0.0000
Degrees of freedom: 16

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between TimeofDay and Fear/Greed categories.


In [ ]:
from scipy.stats import chi2_contingency

# Create a contingency table of 'TimeofDay' and 'Fear/Greed'
contingency_table_timeofday_fear_greed = pd.crosstab(df['TimeofDay'], df['Buy/Sell'])

# Print the contingency table
print("Contingency Table (TimeofDay vs. Buy/Sell):")
display(contingency_table_timeofday_fear_greed)

# Perform the chi-square test
chi2_timeofday_fear_greed, p_timeofday_fear_greed, dof_timeofday_fear_greed, expected_timeofday_fear_greed = chi2_contingency(contingency_table_timeofday_fear_greed)

# Print the results of the chi-square test
print("\nChi-square test results (TimeofDay vs. Buy/Sell):")
print(f"Chi-square statistic: {chi2_timeofday_fear_greed:.4f}")
print(f"P-value: {p_timeofday_fear_greed:.4f}")
print(f"Degrees of freedom: {dof_timeofday_fear_greed}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p_timeofday_fear_greed < alpha:
    print(f"The p-value ({p_timeofday_fear_greed:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between TimeofDay and Buy/Sell categories.")
else:
    print(f"The p-value ({p_timeofday_fear_greed:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between TimeofDay and Buy/Sell categories.")

Contingency Table (TimeofDay vs. Buy/Sell):


Buy/Sell,buy,sell
TimeofDay,,
afternoon,19764,21792
evening,21411,24611
lateNight,33810,33836
morning,16800,19076
night,10905,9213



Chi-square test results (TimeofDay vs. Buy/Sell):
Chi-square statistic: 447.1578
P-value: 0.0000
Degrees of freedom: 4

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between TimeofDay and Buy/Sell categories.


In [ ]:
# Prepare data for ANOVA: create a list of Total_Amount for each profit and loss category
mask = ~(df['TradeResult'] == 'nill')
trade_result_group = df[mask].groupby('TradeResult')['Total_Amount'].apply(list)

# Perform one-way ANOVA test
# Note: ANOVA requires that groups have roughly equal variances and the data within each group is approximately normally distributed.
# We proceed with the test, but these assumptions should ideally be checked.
f_statistic, p_value = stats.f_oneway(*trade_result_group)

# Print the results
print("ANOVA test results for Total_Amount across trade result  categories:")
print(f"F-statistic: {f_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpret the results
alpha = 0.05  # Significance level

print("\nInterpretation:")
if p_value < alpha:
    print(f"The p-value ({p_value:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant difference in the mean Total_Amount across profit and loss categories.")
else:
    print(f"The p-value ({p_value:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant difference in the mean Total_Amount across profit and loss categories.")

ANOVA test results for Total_Amount across trade result  categories:
F-statistic: 3.6478
P-value: 0.0561

Interpretation:
The p-value (0.0561) is greater than the significance level (0.05).
We fail to reject the null hypothesis. There is no statistically significant difference in the mean Total_Amount across profit and loss categories.


In [ ]:
# Create a contingency table of 'TimeofDay' and 'Fear/Greed'
ct = pd.crosstab(df['TradeResult'], df['Fear/Greed'])


display(ct)

# Perform the chi-square test
chi2_st, p_, dof_, expected_time = chi2_contingency(contingency_table_timeofday_fear_greed)

# Print the results of the chi-square test
print("\nChi-square test results (Profit/Loss vs. Fear/Greed):")
print(f"Chi-square statistic: {chi2_st:.4f}")
print(f"P-value: {p_:.4f}")
print(f"Degrees of freedom: {dof_}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p_ < alpha:
    print(f"The p-value ({p_:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between  'TimeofDay' and 'Fear/Greed categories.")
else:
    print(f"The p-value ({p_:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between  'TimeofDay' and 'Fear/Greed categories.")

Fear/Greed,extreme fear,extreme greed,fear,greed,neutral
TradeResult,,,,,
loss,2475,2259,3789,5818,3198
nill,10994,19139,32029,25127,19527
profit,7931,18594,26019,19358,14961



Chi-square test results (Profit/Loss vs. Fear/Greed):
Chi-square statistic: 447.1578
P-value: 0.0000
Degrees of freedom: 4

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between  'TimeofDay' and 'Fear/Greed categories.


In [ ]:
from scipy.stats import chi2_contingency
pnl_df = df[mask]
# Create a contingency table of 'TimeofDay' and crossed
ct = pd.crosstab(pnl_df['TradeResult'], pnl_df['Crossed'])


display(ct)

# Perform the chi-square test
chi2_st, p_, dof_, expected_time = chi2_contingency(contingency_table_timeofday_fear_greed)

# Print the results of the chi-square test
print("\nChi-square test results (Profit/Loss vs. Crossed):")
print(f"Chi-square statistic: {chi2_st:.4f}")
print(f"P-value: {p_:.4f}")
print(f"Degrees of freedom: {dof_}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p_ < alpha:
    print(f"The p-value ({p_:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between Profit/Loss and Crossed(true/False) categories.")
else:
    print(f"The p-value ({p_:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between profit/loss  and Crossed(True/False) categories.")

Crossed,False,True
TradeResult,,
loss,5807,11732
profit,34882,51981



Chi-square test results (Profit/Loss vs. Crossed):
Chi-square statistic: 447.1578
P-value: 0.0000
Degrees of freedom: 4

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between Profit/Loss and Crossed(true/False) categories.


In [ ]:
# Calculate the average profit for each user across all times of day
average_profit_by_user = df[df['PnL'] > 0] \
    .groupby('User_Id')['PnL'].mean().sort_values(ascending=False)

# Get the top 5 users by average profit
top_10_profit_users = average_profit_by_user.head(10).index.tolist()

# Filter the profitable trades data for the top 5 profit users
top_profit_trades = df[(df['PnL'] > 0) &
                              (df['User_Id'].isin(top_10_profit_users))].copy()

top_loss_trades = df[(df['PnL'] <0) &
                              (df['User_Id'].isin(top_10_profit_users))].copy()

In [ ]:
trades_by_top_10_traders = pd.concat([top_profit_trades, top_loss_trades])
trades_by_top_10_traders.shape

(13818, 20)

In [ ]:
all_trades_top_10= df[df['User_Id'].isin(top_10_profit_users)].copy()
all_trades_top_10.shape

(30931, 20)

In [ ]:
df.shape

(211218, 20)

In [ ]:
from scipy.stats import chi2_contingency

ct = pd.crosstab(all_trades_top_10['Coin_Type'], all_trades_top_10['Fear/Greed'])


display(ct)

# Perform the chi-square test
chi2_st, p_, dof_, expected_time = chi2_contingency(contingency_table_timeofday_fear_greed)

# Print the results of the chi-square test
print("\nChi-square test results (choice of Coin  vs. Fear/Greed): by top 5 profitable traders when they made profit")
print(f"Chi-square statistic: {chi2_st:.4f}")
print(f"P-value: {p_:.4f}")
print(f"Degrees of freedom: {dof_}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p_ < alpha:
    print(f"The p-value ({p_:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between Choice of Coin and Fear/greed) categories.")
else:
    print(f"The p-value ({p_:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between Choice of Coin and Fear/greed) categories.")

Fear/Greed,extreme fear,extreme greed,fear,greed,neutral
Coin_Type,,,,,
expensive,1225,1331,3606,4134,1199
moderate,1170,1652,3145,2515,3055
penny,711,992,1809,3354,1033



Chi-square test results (choice of Coin  vs. Fear/Greed): by top 5 profitable traders when they made profit
Chi-square statistic: 447.1578
P-value: 0.0000
Degrees of freedom: 4

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between Choice of Coin and Fear/greed) categories.


In [ ]:
ct = pd.crosstab(top_profit_trades['Coin_Type'], top_profit_trades['Fear/Greed'])


display(ct)

# Perform the chi-square test
chi2_st, p_, dof_, expected_time = chi2_contingency(contingency_table_timeofday_fear_greed)

# Print the results of the chi-square test
print("\nChi-square test results (choice of Coin  vs. Fear/Greed): by top 5 profitab;e traders when they incurred loss")
print(f"Chi-square statistic: {chi2_st:.4f}")
print(f"P-value: {p_:.4f}")
print(f"Degrees of freedom: {dof_}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p_ < alpha:
    print(f"The p-value ({p_:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between Choice of Coin and Fear/greed) categories.")
else:
    print(f"The p-value ({p_:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between Choice of Coin and Fear/greed) categories.")

Fear/Greed,extreme fear,extreme greed,fear,greed,neutral
Coin_Type,,,,,
expensive,588,106,1631,1049,621
moderate,315,853,1318,1160,1457
penny,193,278,569,1111,142



Chi-square test results (choice of Coin  vs. Fear/Greed): by top 5 profitab;e traders when they incurred loss
Chi-square statistic: 447.1578
P-value: 0.0000
Degrees of freedom: 4

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between Choice of Coin and Fear/greed) categories.


In [ ]:
ct = pd.crosstab(all_trades_top_10['Buy/Sell'], all_trades_top_10['Fear/Greed'])


display(ct)

# Perform the chi-square test
chi2_st, p_, dof_, expected_time = chi2_contingency(contingency_table_timeofday_fear_greed)

# Print the results of the chi-square test
print("\nChi-square test results (choice of Buy/Sell  vs. Fear/Greed): by top 5 profitab;e traders when they made profit")
print(f"Chi-square statistic: {chi2_st:.4f}")
print(f"P-value: {p_:.4f}")
print(f"Degrees of freedom: {dof_}")

# Interpret the results
alpha = 0.05 # Significance level

print("\nInterpretation:")
if p_ < alpha:
    print(f"The p-value ({p_:.4f}) is less than the significance level ({alpha}).")
    print("We reject the null hypothesis. There is a statistically significant association between choice of Buy/Sell  vs. Fear/Greed categories.")
else:
    print(f"The p-value ({p_:.4f}) is greater than the significance level ({alpha}).")
    print("We fail to reject the null hypothesis. There is no statistically significant association between choice of Buy/Sell  vs. Fear/Greed categories.")

Fear/Greed,extreme fear,extreme greed,fear,greed,neutral
Buy/Sell,,,,,
buy,1621,1138,4573,3772,2578
sell,1485,2837,3987,6231,2709



Chi-square test results (choice of Buy/Sell  vs. Fear/Greed): by top 5 profitab;e traders when they made profit
Chi-square statistic: 447.1578
P-value: 0.0000
Degrees of freedom: 4

Interpretation:
The p-value (0.0000) is less than the significance level (0.05).
We reject the null hypothesis. There is a statistically significant association between choice of Buy/Sell  vs. Fear/Greed categories.


In [ ]:
# Ensure TimeofDay is categorical
trades_by_top_10_traders['TimeofDay'] = all_trades_top_10['TimeofDay'].astype('category')

# Group data
groups = [grp['PnL'].values for _, grp in trades_by_top_10_traders.groupby('TimeofDay') if len(grp) >= 3]

# ✅ Assumption 1: Shapiro-Wilk Test for Normality
print("📌 Shapiro-Wilk Normality Test (PnL by TimeofDay):")
for name, grp in trades_by_top_10_traders.groupby('TimeofDay'):
    if len(grp) >= 3:
        stat, p = shapiro(grp['PnL'])
        print(f"  {name}: W = {stat:.3f}, p = {p:.3f} → {'✅ Normal' if p > 0.05 else '❌ Not Normal'}")

# ✅ Assumption 2: Levene’s Test for Homogeneity of Variance
stat_levene, p_levene = levene(*groups)
print(f"\n📌 Levene’s Test for Equal Variances: Stat = {stat_levene:.3f}, p = {p_levene:.3f} → {'✅ Equal variances' if p_levene > 0.05 else '❌ Unequal variances'}")

# ✅ One-Way ANOVA Test
f_stat, p_val = f_oneway(*groups)
print(f"\n📊 One-Way ANOVA: F = {f_stat:.3f}, p = {p_val:.4f}")
print("🔍 Result:", "✅ Significant difference in mean PnL across TimeofDay" if p_val < 0.05 else "❌ No significant difference in mean PnL across TimeofDay")


📌 Shapiro-Wilk Normality Test (PnL by TimeofDay):
  afternoon: W = 0.139, p = 0.000 → ❌ Not Normal
  evening: W = 0.198, p = 0.000 → ❌ Not Normal
  lateNight: W = 0.167, p = 0.000 → ❌ Not Normal
  morning: W = 0.247, p = 0.000 → ❌ Not Normal
  night: W = 0.155, p = 0.000 → ❌ Not Normal

📌 Levene’s Test for Equal Variances: Stat = 8.270, p = 0.000 → ❌ Unequal variances

📊 One-Way ANOVA: F = 9.967, p = 0.0000
🔍 Result: ✅ Significant difference in mean PnL across TimeofDay


/tmp/ipython-input-117-2540174197.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = [grp['PnL'].values for _, grp in trades_by_top_10_traders.groupby('TimeofDay') if len(grp) >= 3]
/tmp/ipython-input-117-2540174197.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for name, grp in trades_by_top_10_traders.groupby('TimeofDay'):
/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 5245.
  res = hypotest_fun_out(*samples, **kwds)


## Statistical Analysis Summary and Conclusion

Based on the statistical tests performed:

**ANOVA Test for PnL across TimeofDay (All Users):**

*   The ANOVA test showed a statistically significant difference in the mean Profit and Loss (PnL) across different times of the day for all users (p-value = 0.0000).
*   However, the assumption checks for ANOVA (Shapiro-Wilk for normality and Levene's for homogeneity of variances) failed (p-values < 0.05 for all groups and Levene's test). This indicates that the data is not normally distributed within each time of day group and the variances are not equal across groups.
*   **Conclusion:** Due to the violation of ANOVA assumptions, the results of the ANOVA test for all users should be interpreted with caution. A non-parametric alternative, such as the Kruskal-Wallis test, would be more appropriate to compare the median PnL across different times of the day.

**Chi-Square Test for Buy/Sell vs. Fear/Greed (All Users):**

*   The chi-square test showed a statistically significant association between Buy/Sell decisions and Fear/Greed categories (p-value = 0.0000).

**Chi-Square Test for Crossed vs. Fear/Greed (All Users):**

*   The chi-square test showed a statistically significant association between Crossed status and Fear/Greed categories (p-value = 0.0000).

**ANOVA Test for Total_Amount across Fear/Greed (All Users):**

*   The ANOVA test showed a statistically significant difference in the mean Total\_Amount across the Fear/Greed categories (p-value = 0.0000).

**Chi-Square Test for TimeofDay vs. Fear/Greed (All Users):**

*   The chi-square test showed a statistically significant association between TimeofDay and Fear/Greed categories (p-value = 0.0000).

**Chi-Square Test for TimeofDay vs. Buy/Sell (All Users):**

*   The chi-square test showed a statistically significant association between TimeofDay and Buy/Sell categories (p-value = 0.0000).

**ANOVA Test for Total_Amount across TradeResult (Excluding 'nill') (All Users):**

*   The ANOVA test did **not** show a statistically significant difference in the mean Total\_Amount across profit and loss categories (p-value = 0.0561).

**Chi-Square Test for TradeResult vs. Fear/Greed (Excluding 'nill') (All Users):**

*   The chi-square test showed a statistically significant association between TradeResult and Fear/Greed categories (p-value = 0.0000).

**Chi-Square Test for TradeResult vs. Crossed (Excluding 'nill') (All Users):**

*   The chi-square test showed a statistically significant association between TradeResult and Crossed (true/false) categories (p-value = 0.0000).

**Analysis for Top 10 Profitable Users:**

*   **ANOVA Test for PnL across TimeofDay (Top 10 Profitable Users, Profitable Trades):**
    *   The ANOVA test showed a statistically significant difference in the mean PnL across different times of the day for profitable trades of the top 10 profitable users (p-value = 0.0000).
    *   However, the assumption checks for ANOVA (Shapiro-Wilk for normality and Levene's for homogeneity of variances) failed (p-values < 0.05 for all groups and Levene's test). This indicates that the data is not normally distributed within each time of day group and the variances are not equal across groups for these specific trades.
    *   **Conclusion:** Similar to the analysis for all users, the violation of ANOVA assumptions means the results for the top 10 profitable users' profitable trades should be interpreted with caution. A non-parametric test is recommended.

*   **Chi-Square Test for Coin\_Type vs. Fear/Greed (Top 10 Profitable Users):**
    *   The chi-square test showed a statistically significant association between the choice of Coin Type and Fear/Greed categories for the top 10 profitable users (p-value = 0.0000).

*   **Chi-Square Test for Buy/Sell vs. Fear/Greed (Top 10 Profitable Users):**
    *   The chi-square test showed a statistically significant association between Buy/Sell decisions and Fear/Greed categories for the top 10 profitable users (p-value = 0.0000).

**Overall Conclusion regarding ANOVA:**

Based on the Shapiro-Wilk and Levene's tests, the assumption of normality and homogeneity of variances required for ANOVA are not met for the PnL data across TimeofDay categories, both for all users and for the profitable trades of the top 10 profitable users. Therefore, the results of the ANOVA tests for PnL across TimeofDay should be considered with caution, and non-parametric tests would be more appropriate for further analysis of this relationship.